In [6]:
import os
import pickle
import numpy as np

In [26]:
# poor man's data loader
corpus_type = 'books'

train_pt_file = f'data/{corpus_type}_train.pt'
val_pt_file   = f'data/{corpus_type}_val.pt'
meta_pt_file  = f'data/{corpus_type}_meta.pkl'
model_pt_file = f'model/{corpus_type}.pt'
model_finetuned_pt_file = f'model/{corpus_type}.pt'

train_data = torch.load(train_pt_file)
val_data   = torch.load(val_pt_file)

tensor([  0,  85, 113, 117, 115, 118, 103, 112, 107, 109])

In [27]:
# attempt to derive vocab_size from the dataset
vocab_size = None
if os.path.exists(meta_pt_file):
    with open(meta_pt_file, 'rb') as f:
        meta = pickle.load(f)
    vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_pt_file})")
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])


found vocab_size = 132 (inside data/meta.pkl)


In [84]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 256 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 50_000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

# # here are all the unique characters that occur in this text
# # chars = sorted(list(set(text)))
# vocab_size = len(chars)
# # create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# # Train and test splits
# data = torch.tensor(encode(text), dtype=torch.long)
# n = int(0.9*len(data)) # first 90% will be train, rest val
# train_data = data[:n]
# val_data = data[n:]

# data loading
def get_main_task_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

def get_batch(split):
    return get_main_task_batch(split)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens, hard_seq = False):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            if hard_seq == True:
                idx_next = torch.argmax(probs, dim=1).reshape(1,-1) # (B, 1)
            else:
                idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
    
model = BigramLanguageModel()
print(device)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

cuda
10.766724 M parameters


In [29]:
# training loop
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 5.0441, val loss 5.0465
step 100: train loss 2.6633, val loss 2.6815
step 200: train loss 2.2416, val loss 2.2667
step 300: train loss 1.9490, val loss 1.9736
step 400: train loss 1.7706, val loss 1.8002
step 500: train loss 1.6615, val loss 1.6843
step 600: train loss 1.5880, val loss 1.6135
step 700: train loss 1.5351, val loss 1.5513
step 800: train loss 1.4959, val loss 1.5165
step 900: train loss 1.4683, val loss 1.4764
step 1000: train loss 1.4352, val loss 1.4537
step 1100: train loss 1.4139, val loss 1.4289
step 1200: train loss 1.3980, val loss 1.4089
step 1300: train loss 1.3781, val loss 1.3878
step 1400: train loss 1.3661, val loss 1.3759
step 1500: train loss 1.3490, val loss 1.3567
step 1600: train loss 1.3400, val loss 1.3480
step 1700: train loss 1.3276, val loss 1.3395
step 1800: train loss 1.3119, val loss 1.3239
step 1900: train loss 1.3095, val loss 1.3172
step 2200: train loss 1.2838, val loss 1.2918
step 2300: train loss 1.2796, val loss 1.2866


In [30]:
if not os.path.exists('model'):
    !mkdir model/
    torch.save(m.state_dict(), model_pt_file)

In [198]:
# Finetune: see Finetuning is no different than training, we just make sure to initialize from a pretrained model and train with a smaller learning rate
max_iters = 20_000
# finetune at constant LR
learning_rate = 3e-5
decay_lr = False

model = BigramLanguageModel()
print(device)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

m.load_state_dict(torch.load(model_pt_file))

cuda
10.766724 M parameters


<All keys matched successfully>

In [199]:
PAD = ' '
STOP = '.'
REVERSE_TOK = torch.tensor(encode('ПФ='), dtype=torch.long)
PAREM_TOK = torch.tensor(encode('ПП='), dtype=torch.long)
PAD_TOK = torch.tensor(encode(PAD), dtype=torch.long)
STOP_TOK = torch.tensor(encode(STOP), dtype=torch.long)

In [200]:
parems_list = []
with open('data/parems_cleaned.txt') as f:
    for line in f:
        parems_list.append(line.strip('\r\n'))
        
n = int(0.9*len(parems_list)) # first 90% will be train, rest val
indicies = [i for i in range(len(parems_list))]
np.random.shuffle(parems_list)
# parems_list = parems_list[]
parems_train_data = parems_list[:n]
parems_val_data   = parems_list[n:]

In [201]:
def get_parems_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = parems_train_data if split == 'train' else parems_val_data
    # take random parem
    ix = torch.randint(0, len(data), (1, ))
    sent = torch.tensor(encode(' '.join([data[i] for i in ix])), dtype=torch.long)
    # divide it randomly - may not from 1????
    j = torch.randint(1, len(sent), (1,))
#     print(sent)
    t = torch.cat([
        sent[:j],
        PAREM_TOK, 
        sent[j:],
        STOP_TOK,
        PAD_TOK.repeat(max(0,2*block_size - len(sent) - 4))
    ], dim=0) 
#     print(t.shape)
    bs = min(len(sent)+j, block_size)
    x = torch.stack([t[i:i+block_size] for i in range(bs)])
    y = torch.stack([t[i+1:i+1+block_size] for i in range(bs)])
    x, y = x.to(device), y.to(device)
    return x, y

In [203]:
def get_reverse_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    i = torch.randint(len(data) - block_size, (1, ))
    j = torch.randint(block_size//2,  block_size, (1,))
    sent = data[i:i+j]
    sent_len = len(sent)
    t = torch.cat([
        sent, 
        REVERSE_TOK, 
#         SEP_TOK, 
        torch.flip(sent, dims=(0,)),
        STOP_TOK,
        PAD_TOK.repeat(max(0,2*block_size - 2*sent_len - 4))
    ], dim=0) 
    # print(input_tensor)
    # print(output_tensor)
#     print(t)
    x = torch.stack([t[i:i+block_size] for i in range(block_size)])
    y = torch.stack([t[i+1:i+1+block_size] for i in range(block_size)])
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    x, y = x.to(device), y.to(device)
    return x, y

In [204]:
def get_batch(split):
    task0_x, task0_y = get_main_task_batch(split) # не забываем про основную задачу
    task1_x, task1_y = get_reverse_batch(split)   # учим на переворот
    task2_x, task2_y = get_parems_batch(split)    # учим на пословицы
    return torch.cat([task0_x, task1_x, task2_x], dim=0), torch.cat([task0_y, task1_y, task2_y], dim=0) 

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 1.5985, val loss 1.6149
step 100: train loss 1.2622, val loss 1.2727
step 200: train loss 1.2079, val loss 1.2300
step 300: train loss 1.1858, val loss 1.1985
step 400: train loss 1.1841, val loss 1.2007
step 500: train loss 1.1692, val loss 1.1918
step 600: train loss 1.1620, val loss 1.1858
step 700: train loss 1.1649, val loss 1.1786
step 800: train loss 1.1562, val loss 1.1698
step 900: train loss 1.1520, val loss 1.1700
step 1000: train loss 1.1512, val loss 1.1551
step 1100: train loss 1.1424, val loss 1.1529
step 1200: train loss 1.1397, val loss 1.1556
step 1300: train loss 1.1347, val loss 1.1488
step 1400: train loss 1.1221, val loss 1.1497
step 1500: train loss 1.1279, val loss 1.1392
step 1600: train loss 1.1250, val loss 1.1447
step 1700: train loss 1.1224, val loss 1.1381
step 1800: train loss 1.1184, val loss 1.1298
step 1900: train loss 1.1199, val loss 1.1280
step 2000: train loss 1.1193, val loss 1.1268
step 2100: train loss 1.1186, val loss 1.1243


In [ ]:
sent = parems_list[12][:30] #decode(data[0:0+32].tolist())
context = torch.cat([
#     REVERSE_TOK, 
    torch.tensor(encode(sent), dtype=torch.long),
    REVERSE_TOK, 
    ], dim=0).reshape(1,-1).to(device)# torch.zeroes((1,1), dtype=torch.long, device=device)
output = decode(m.generate(context, max_new_tokens=len(sent), hard_seq = True)[0].tolist()) + "]"
print(sent)
print(output)

In [ ]:
sent = 'Байден '[:32] #decode(data[0:0+32].tolist())
context = torch.cat([
#     PAREM_TOK,
    torch.tensor(encode(sent), dtype=torch.long),
    PAREM_TOK,
    ], dim=0).reshape(1,-1).to(device)# torch.zeroes((1,1), dtype=torch.long, device=device)
output = decode(m.generate(context, max_new_tokens=64, hard_seq=True)[0].tolist()) + "]"
print(sent)
print(output)

In [ ]:
torch.save(m.state_dict(), model_finetuned_pt_file)

In [ ]:
def inference(sent):
    context = torch.cat([
    #     REVERSE_TOK, 
        torch.tensor(encode(sent), dtype=torch.long),
#         REVERSE_TOK, 
        ], dim=0).reshape(1,-1).to(device)# torch.zeroes((1,1), dtype=torch.long, device=device)
    output = decode(m.generate(context, max_new_tokens=100, hard_seq = False)[0].tolist())
    stop_pos = output.rfind('.')
    if stop_pos > 2*len(sent):
        output = output[:stop_pos]
    return output

test_cases = [
    'как говорит радио Москвы',
    'Был бы ум, а ПП=',
    'Куда ни глянь везде ПФ='
]

for sent in test_cases:
    output = inference(sent)
    print("Prompt : ", sent)
    print("Predict: ", output)